This Python script is designed to interact with the Riot Games API for fetching League of Legends game data. Here's a quick overview of what each part does:

- **Imports**: Necessary libraries like `os` for environment variables, `json` for data handling, `time` for adding delays, `requests` for web requests, and `dotenv` for loading environment variables from a `.env` file.

- **Environment Setup**: Loads API keys and other sensitive data from a `.env` file to keep them secure.

- **Configuration**: Sets up several constants to be used with the Riot Games API:
  - `region` and `continent` for location-specific data.
  - `queue_id` and `queue` for filtering games by type.
  - `number_of_games` and `number_of_players` to control the amount of data fetched.
  - `sleep_time` to prevent exceeding the API's rate limit.
  - `api_key` to authenticate the requests.

The script sets the stage for making targeted requests to the Riot Games API to obtain game and player data while handling rate limits and ensuring secure access to the API key.

In [101]:
import os
import json
import time
import requests
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Constant values
region = 'kr'
queue_id = 420
sleep_time = 0.5
continent = 'asia'
number_of_games = 100
number_of_players = 200
queue = 'RANKED_SOLO_5x5'
api_key = os.environ.get('RIOT_API_KEY')

This section of code works with data related to League of Legends champions:

- **Loading Champion Data**: It starts by loading champion information from a local JSON file named `champion_data.json`. This file is expected to contain champion data under a `"data"` key.

- **Creating a Champion Hashmap**: A dictionary (`champion_hashmap`) is created to map champion IDs (`key`) to their names. This makes it easier to look up champion names based on their unique IDs.

- **Saving the Hashmap**: The `champion_hashmap` is then saved to another JSON file called `champion_hashmap.json`. This file stores the mapping in a structured format, which can be easily accessed for future reference.

- **Function to Retrieve the Hashmap**: The `get_champion_hashmap` function is defined to load and return the champion ID-to-name mapping from the `champion_hashmap.json` file whenever it's called. This function simplifies accessing the champion hashmap elsewhere in the script or application.

This approach allows for efficient management and access to champion data, particularly useful for applications requiring quick lookups of champion names based on IDs.

In [56]:
champion_data = {}
with open('champion_data.json', 'r') as file:
    champion_data = json.load(file)['data']

champion_hashmap = { champion_data[key]['key']: key for key in champion_data }

with open('champion_hashmap.json', 'w') as file:
    json.dump(champion_hashmap, file, indent=4)

def get_champion_hashmap():
    data = {}
    with open('champion_hashmap.json', 'r') as file:
        data = json.load(file)
    return data

- **`read_json_data` Function**:
  - This function is responsible for reading and returning the contents of `league_korean_data.json`. It opens the file in read mode, loads its JSON content into a dictionary, and returns this dictionary. This is useful for accessing the stored data for further processing or analysis.

- **`update_json_data` Function**:
  - Takes three arguments: the current data (as a dictionary), a `key`, and a `value` to update the data with.
  - This function updates the dictionary with the new key-value pair provided as arguments. Then, it writes the updated dictionary back to `league_korean_data.json`, effectively saving the changes to the file. The `indent=4` parameter makes the output file human-readable by adding whitespace to the JSON structure.
  - It's designed to modify or add new entries to the JSON data, allowing dynamic updates to the stored data based on the application's needs or user input.

These functions facilitate easy reading and updating of JSON data, making it straightforward to manage the storage and retrieval of League of Legends data for Korean players.

In [40]:
def read_json_data():
    data = {}
    with open('league_korean_data.json', 'r') as file:
        data = json.load(file)
    return data

def update_json_data(data, key, value):
    data[key] = value

    with open('league_korean_data.json', 'w') as file:
        json.dump(data, file, indent=4)
    

- **Fetching Challenger League Data**:
  - Constructs a URL to access the challenger league data from the Riot Games API. The URL includes the selected region and queue type, both of which are defined earlier in the script, and the API key for authentication.
  - Makes a GET request to the constructed URL to fetch the data.

- **Processing the Response**:
  - Checks if the request was successful (`status_code == 200`).
  - Parses the JSON response to extract the `entries`, which contain information about the players in the challenger league.
  - Sorts these entries based on the players' league points (LP) in descending order, ensuring the list starts with the player having the highest LP.

- **Extracting Top Player IDs**:
  - From the sorted list, extracts the summoner IDs of the top 200 players. This step focuses on identifying these top players for further analysis or processing.

- **Updating the JSON Data**:
  - Calls the previously defined `update_json_data` function, creating or updating a JSON file named `league_korean_data.json` to store the summoner IDs of these top players under the key `'players'`.
  
This snippet is particularly useful for applications or analyses focusing on the top performers in the League of Legends challenger league, enabling targeted data collection and insights into high-level play dynamics.

In [41]:

url = 'https://{}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{}?api_key={}'.format(region, queue, api_key)

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    entries = data['entries']
    
    # Sort the players by their league points (LP) in descending order
    sorted_entries = sorted(entries, key=lambda x: x['leaguePoints'], reverse=True)
    
    # Get the top 50 players' summoner ids
    top_player_ids = [entry['summonerId'] for entry in sorted_entries[:200]]

    update_json_data({}, 'players', top_player_ids)


- **Reading Existing Data**: The `read_json_data` function is called to load data from `league_korean_data.json`, particularly focusing on the `players` key that contains summoner IDs.

- **Initializing a Set for Summoner UUIDs**: A set named `summoner_uuids` is created to store unique player UUIDs (PUUIDs), ensuring there are no duplicates.

- **Fetching Summoner UUIDs**:
  - Iterates over each summoner ID from the previously loaded data.
  - For each ID, constructs a URL to fetch the summoner's detailed information from the Riot Games API, including their PUUID.
  - If the request is successful, adds the summoner's PUUID to the `summoner_uuids` set.
  - Implements a delay (`sleep_time`) after each request to avoid triggering the API's rate limit.

- **Updating the JSON Data**: Finally, updates `league_korean_data.json` with the new data:
  - Converts the `summoner_uuids` set to a list (since sets are not JSON serializable) and stores it under the key `'uuids'` in the existing data.
  - Uses the `update_json_data` function to write these changes back to the file.

This process effectively enriches the previously fetched challenger league data with unique identifiers for each player, facilitating more detailed or personalized data retrieval and analysis in subsequent steps.

In [45]:
existing_data = read_json_data()
summoner_ids = existing_data['players']

summoner_uuids = set()

for summoner_id in summoner_ids:
    url = 'https://{}.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(region, summoner_id, api_key)
    response = requests.get(url)

    if response.status_code == 200:
        summoner_uuids.add(response.json().get('puuid'))

    # Prevent rate limiter
    time.sleep(sleep_time)

update_json_data(existing_data, 'uuids', list(summoner_uuids))

- **Loading Existing Data**: Uses the `read_json_data` function to load data from `league_korean_data.json`, focusing on the `'uuids'` key to get a list of summoner UUIDs.

- **Initializing a Set for Game IDs**: Creates a set named `game_ids` to store unique game IDs, ensuring no duplicates.

- **Fetching Game IDs**:
  - Iterates through each summoner UUID.
  - Constructs two URLs to fetch a specified number of game IDs (`number_of_games`) associated with the summoner's PUUID from the Riot Games API. The first URL fetches the initial batch of game IDs, and the second URL fetches another batch starting from the end of the first batch, effectively doubling the data retrieved per summoner.
  - If the requests are successful, updates the `game_ids` set with the IDs returned from the API.
  - Adds a delay (`sleep_time`) after each request to respect the API's rate limit.

- **Updating the JSON Data**:
  - Converts the `game_ids` set to a list (to ensure JSON serializability) and updates the existing JSON data with this list under the key `'game_ids'`.
  - Calls the `update_json_data` function to save these changes back to `league_korean_data.json`.

This approach methodically collects game IDs for further analysis or processing, significantly expanding the dataset with valuable identifiers for individual matches. It's a crucial step for applications or analyses requiring in-depth match histories or patterns across numerous games.

In [46]:
existing_data = read_json_data()
summoner_uuids = existing_data['uuids']

game_ids = set()

for summoner_uuid in summoner_uuids:
    url = 'https://{}.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?api_key={}&count={}'.format(
        continent, summoner_uuid, api_key, number_of_games
    )
    response = requests.get(url)

    if response.status_code == 200:
        game_ids.update(response.json())

    # Prevent rate limiter
    time.sleep(sleep_time)

    url = 'https://{}.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?api_key={}&start={}&count={}'.format(
        continent, summoner_uuid, api_key, number_of_games, number_of_games
    )
    response = requests.get(url)

    if response.status_code == 200:
        game_ids.update(response.json())
    
    # Prevent rate limiter
    time.sleep(sleep_time)

update_json_data(existing_data, 'game_ids', list(game_ids))

This code snippet is focused on generating a dataset from League of Legends match data, with an emphasis on champion picks, bans, and match outcomes. Below is a detailed breakdown of its functionality:

### Preparing the Data:
- The script begins by reading previously gathered game IDs and loading a champion hashmap, which maps champion IDs to their corresponding names.

### Setting Up the CSV File:
- It creates a new CSV file named `dataset.csv` to store the extracted match data. The headers for this file are dynamically generated to include champion picks for both the blue and red teams across all roles (Top, Jungle, Mid, Bot, Support), their bans, and a column to indicate whether the blue team won.

### Fetching and Processing Match Details:
- For each game ID, the script fetches detailed match information from the Riot Games API, ensuring it only proceeds with matches that have complete data.
- It processes the fetched data to:
  - Identify and record champion bans for both teams.
  - Match each player's chosen champion to their role and team, utilizing predefined mappings for team colors and roles.
  - Exclude any matches that lack complete role data, maintaining the integrity of the dataset.

### Creating and Writing Data Rows:
- Generates a row for each match where:
  - Each cell corresponds to a specific champion-role combination or a ban, marked '1' if applicable (champion was picked/banned) and '0' otherwise.
  - The final cell in each row indicates whether the blue team won the match ('1' for a win, '0' for a loss).
- These rows are then appended to `dataset.csv`, compiling a detailed dataset suitable for further analysis.

### Rate Limit Compliance:
- Implements pauses between API requests to comply with rate limiting, preventing the script from being blocked for excessive querying.

This crafted dataset offers a rich source for analyzing game strategies, effectiveness of champion-role combinations, and the strategic impact of bans on the outcomes of League of Legends matches.

In [ ]:
existing_data = read_json_data()
game_ids = existing_data['game_ids']
champion_hashmap = get_champion_hashmap()

headerKeys = [
    'blueTop', 'blueJg', 'blueMid', 'blueBot', 'blueSupp',
    'redTop', 'redJg', 'redMid', 'redBot', 'redSupp',
    'blueBan', 'redBan'
]

with open('dataset.csv', 'w') as file:
    headers = []
    for typekey in headerKeys:
        for champion in champion_hashmap.values():
            headers.append(typekey + champion)
    headers.append('blueWin')

    file.write(','.join(headers) + '\n')

    for game_id in game_ids:
        url = 'https://{}.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(
            continent, game_id, api_key
        )
        response = requests.get(url)
        
        if (response.status_code != 200):
            print("request issue")
            time.sleep(sleep_time)
            continue

        match_info = response.json()['info']
        teams = match_info['teams']
        players = match_info['participants']

        if (len(teams) < 2 or len(players) < 10):
            print('missing data')
            time.sleep(sleep_time)
            continue

        # Blue team has id of 100, red has 200
        blue = 0
        red = 1
        if teams[0]['teamId'] == 200:
            blue = 1
            red = 0

        trueColumns = []

        for ban in teams[blue]['bans']:
            champId = ban['championId']
            if champId == -1:
                continue
            trueColumns.append('blueBan' + champion_hashmap[str(champId)])

        for ban in teams[red]['bans']:
            champId = ban['championId']
            if champId == -1:
                continue
            trueColumns.append('redBan' + champion_hashmap[str(champId)])


        team_hash = {
            100: 'blue',
            200: 'red',
        }

        role_hash = {
            'TOP': 'Top',
            'JUNGLE': 'Jg',
            'MIDDLE': 'Mid',
            'BOTTOM': 'Bot',
            'UTILITY': 'Supp'
        }

        invalid = False

        for player in players:
            if (player['teamPosition'] not in role_hash):
                invalid = True
                break
            trueColumns.append(
                team_hash[player['teamId']] + role_hash[player['teamPosition']] + player['championName']
            )

        if (invalid):
            time.sleep(sleep_time)
            continue

        row = []
        for i in range(len(headers) - 1):
            if headers[i] in trueColumns:
                row.append('1')
            else:
                row.append('0')
        
        # Add win
        if (teams[blue]['win']):
            row.append('1')
        else:
            row.append('0')
        
        file.write(','.join(row) + '\n')

        # Prevent rate limiting
        time.sleep(sleep_time)
